ADD EXPLANATION HERE

## Load Packages

In [2]:
import pandas as pd
import numpy as np
import pickle
import os

Get Meta-Data and store separaetly

## Define Functions

### General Parameter

In [70]:
FOLDER = "../data/surveys/"

COLS_META = ["sex", "age", "currentCountry",  "political_opinion", "race", 
        "religion", "education", "income", "social_class"] 

# adjust to survey (can have different scales)
# more information: Intro about our study, explaining what it is about
ALT1 = "For a scientific study, we are collecting responses to various psychological questionnaires. In the following you will be shown statements from a personality test. Please indicate your level of agreement with each statement. There are no right or wrong answers. Use numbers from 1 to 5, with 1 meaning strongly disagree and 5 meaning strongly agree. Respond with a single number. The statement is: "

# semantics/rephraming, replace as much with synonyms, but kept general sense vague
ALT2 = "For an ongoing research project, please express how much you agree with the statement shown at the end. Respond with an integer between 1 and 5, where 1 indicates a strong disagreement and 5 indicates a strong agreement. Respond with a single digit only. Here's the statement:"

#response values (scale)
ALT4 = "For a scientific study, please indicate your level of agreement with the following statement. Use integer numbers from -2 to 2, with -2 meaning strongly disagree and 2 meaning strongly agree. Respond with a single number. The statement is: "

# also test order of items shown
PROMPT_TEXT = "For a scientific study, please indicate your level of agreement with the following statement. Use integer numbers from 1 to 5, with 1 meaning strongly disagree and 5 meaning strongly agree. Respond with a single number. The statement is: "

### Functions

In [68]:
def extractSurveys(data):
    path = FOLDER + data + ".csv"
    df = pd.read_csv(path)
    print(set(COLS_META) <= set(df.columns)) # check if meta information is in dataset
    
    idx = df.columns.tolist().index("sex") #index of last survey item
    cols_items = df.columns.tolist()[:idx]
    print(cols_items[-1]) # print last survey item
    
    df_items = df.dropna(subset=cols_items, axis = 0)
    df_total = df_items.dropna(subset=COLS_META).reset_index(drop=True)
    df_total.to_csv("../data/processed/" + data + "_cleaned.csv", index=False)
    
    item_path = "../data/items/" + data + "_items.csv"
    if not (os.path.isfile(item_path)):
        items_list = pd.DataFrame(cols_items)
        items_list.to_csv(item_path, index=False)
    else:
        pass

    return df_total, cols_items

In [136]:
d = "cognition"
df_cognition, cols_meta_cognition = extractSurveys(d)

True
cognition_18


In [137]:
items = pd.read_csv("../data/items/" + d + "_items.csv", sep=";")
# check the respective texts in the original questionnaires
prompts = [PROMPT_TEXT + "\"" + x + "\"" for x in items.item_text]
#show prompts
with open("../data/prompts/" + d + ".pkl", "wb") as output:
    pickle.dump(prompts, output)

## Closure

In [39]:
d = "closure"
df_closure, cols_meta_closure = extractSurveys(d)

True
closure_16


In [53]:
items = pd.read_csv("../data/items/" + d + "_items.csv", sep=";")
# check the respective texts in the original questionnaires
prompts = [PROMPT_TEXT + "\"" + x + "\"" for x in items.item_text]
#show prompts
with open("../data/prompts/" + d + ".pkl", "wb") as output:
    pickle.dump(prompts, output)

In [41]:
# choose dataset
d = "closure"
# create path
path = "../data/items/" + d + "_items.csv"
path_items = '../data/items/' + d + '_items.csv'

with open ('../data/prompts/' + d + ".pkl", 'rb') as fp:
    prompts = pickle.load(fp)
    
prompts[0]

'For a scientific study, please indicate your level of agreement with the following statement. Use integer numbers from 1 to 6, with 1 meaning strongly disagree and 6 meaning strongly agree. Respond with a single number. The statement is: "In case of uncertainty, I prefer to make an immediate decision, whatever it may be."'

## BIG5

Prompting is tested here only for the BFI but this pipeline can be extended to any survey (simply adapt the prompts for the respective survey andd repeat the analyses)

In [62]:
d = "bigfive"
df_bigfive, cols_meta_bigfive = extractSurveys(d)

C:\Users\suhai\AppData\Local\Temp\ipykernel_16672\1109698091.py:3: DtypeWarning: Columns (53,63,66) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


True
bigfive_44


In [71]:
items = pd.read_csv("../data/items/" + d + "_items.csv", sep=";")
# check the respective texts in the original questionnaires
for i, prompt_text in enumerate([ALT1,ALT2,ALT3,ALT4,ALT5]):  
    prompts = [prompt_text + "\"" + x + "\"" for x in items.item_text]
    #show prompts
    with open("../data/prompts/ALT" + str(i+1) + "_" + d + ".pkl", "wb") as output:
        pickle.dump(prompts, output)

In [20]:
d = "systems_feelings"
df_systems_feelings, cols_meta_systems_feelings = extractSurveys(d)

C:\Users\suhai\AppData\Local\Temp\ipykernel_23280\1109698091.py:3: DtypeWarning: Columns (61,64) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


True
systems_feelings_42


In [21]:
items = pd.read_csv("../data/items/" + d + "_items.csv", sep=";")
# check the respective texts in the original questionnaires
prompts = [PROMPT_TEXT + "\"" + x + "\"" for x in items.item_text]
#show prompts
with open("../data/prompts/" + d + ".pkl", "wb") as output:
    pickle.dump(prompts, output)

In [119]:
d = "cogref"
df_cogref, cols_meta_cogref = extractSurveys(d)

C:\Users\suhai\AppData\Local\Temp\ipykernel_23280\1109698091.py:3: DtypeWarning: Columns (59,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


True
cogref_40


In [122]:
items = pd.read_csv("../data/items/" + d + "_items.csv", sep=";")
# check the respective texts in the original questionnaires
prompts = [PROMPT_TEXT + "\"" + x + "\"" for x in items.item_text]
#show prompts
with open("../data/prompts/" + d + ".pkl", "wb") as output:
    pickle.dump(prompts, output)